In [2]:
from pyspark.sql import SparkSession,SQLContext

spark = SparkSession.builder.master("local").appName("DA").getOrCreate()



In [14]:
from operator import delitem
import os

current_dir = os.getcwd()
# Define the file name
file_name = "sampleIISlogs/u_ex240425.log"

# needs some cleanup activity before reading the file (like removing the comments)
# Construct the file path
file_path = os.path.join(current_dir, file_name)

ODataFrame1 = spark.read.csv(
file_path,
header=True,
sep=' ',
inferSchema=True,
nullValue='null'
)


In [15]:
ODataFrame1.show(5)

+----------+-------------------+----+---------+--------------------+--------------------+------+-----------+----+--------------------+--------------------+---------+------------+---------------+----------+
|      date|               time|s-ip|cs-method|         cs-uri-stem|        cs-uri-query|s-port|cs-username|c-ip|      cs(User-Agent)|         cs(Referer)|sc-status|sc-substatus|sc-win32-status|time-taken|
+----------+-------------------+----+---------+--------------------+--------------------+------+-----------+----+--------------------+--------------------+---------+------------+---------------+----------+
|2024-04-25|2024-04-25 06:22:45| ::1|      GET|                   /|                   -|  8126|          -| ::1|Mozilla/5.0+(Wind...|                   -|      200|           0|              0|      2050|
|2024-04-25|2024-04-25 06:22:45| ::1|      GET|         /js/site.js|v=4q1jwFhaPaZgr8W...|  8126|          -| ::1|Mozilla/5.0+(Wind...|http://localhost:...|      200|           

In [16]:
# had issues with '-' so replaced it with '_'
newdata = [item.replace('-','_').replace('(','_').replace(')','_') for item in ODataFrame1.schema.names ]
newdata

['date',
 'time',
 's_ip',
 'cs_method',
 'cs_uri_stem',
 'cs_uri_query',
 's_port',
 'cs_username',
 'c_ip',
 'cs_User_Agent_',
 'cs_Referer_',
 'sc_status',
 'sc_substatus',
 'sc_win32_status',
 'time_taken']

In [17]:
data = ODataFrame1.toDF(*newdata)

In [18]:
data.createOrReplaceTempView("IISLogs2")
data.printSchema()

root
 |-- date: date (nullable = true)
 |-- time: timestamp (nullable = true)
 |-- s_ip: string (nullable = true)
 |-- cs_method: string (nullable = true)
 |-- cs_uri_stem: string (nullable = true)
 |-- cs_uri_query: string (nullable = true)
 |-- s_port: integer (nullable = true)
 |-- cs_username: string (nullable = true)
 |-- c_ip: string (nullable = true)
 |-- cs_User_Agent_: string (nullable = true)
 |-- cs_Referer_: string (nullable = true)
 |-- sc_status: integer (nullable = true)
 |-- sc_substatus: integer (nullable = true)
 |-- sc_win32_status: integer (nullable = true)
 |-- time_taken: integer (nullable = true)



In [20]:
sqlq = """
SELECT 
    window(
        to_timestamp(date_format(date, 'yyyy-MM-dd') || ' ' || date_format(time, 'HH:mm:ss')), 
        "5 seconds"
    ) as time_window,
    sc_win32_status,
    count(*) as count,
    round(avg(time_taken), 2) as avg_time_taken
FROM 
    IISLogs2
WHERE 
    sc_status = 200
GROUP BY 
    time_window,
    sc_win32_status
ORDER BY 
    time_window
"""
spark.sql(sqlq).show(80, truncate = False)

+------------------------------------------+---------------+-----+--------------+
|time_window                               |sc_win32_status|count|avg_time_taken|
+------------------------------------------+---------------+-----+--------------+
|{2024-04-25 06:22:45, 2024-04-25 06:22:50}|0              |10   |250.2         |
|{2024-04-25 06:22:55, 2024-04-25 06:23:00}|0              |1    |6191.0        |
|{2024-04-25 06:22:55, 2024-04-25 06:23:00}|64             |1    |8632.0        |
|{2024-04-25 06:23:00, 2024-04-25 06:23:05}|0              |2    |6043.0        |
|{2024-04-25 06:23:15, 2024-04-25 06:23:20}|0              |1    |7170.0        |
|{2024-04-25 06:23:30, 2024-04-25 06:23:35}|0              |1    |5023.0        |
|{2024-04-25 06:23:35, 2024-04-25 06:23:40}|0              |1    |10030.0       |
|{2024-04-25 06:23:40, 2024-04-25 06:23:45}|0              |2    |13429.5       |
|{2024-04-25 06:23:45, 2024-04-25 06:23:50}|0              |1    |20044.0       |
|{2024-04-25 06:

In [21]:
spark.sql("""SELECT * from IISlogs2 where cs_uri_stem='/Products/Featured'""").show(5)

+----------+-------------------+----+---------+------------------+------------+------+-----------+----+--------------------+--------------------+---------+------------+---------------+----------+
|      date|               time|s_ip|cs_method|       cs_uri_stem|cs_uri_query|s_port|cs_username|c_ip|      cs_User_Agent_|         cs_Referer_|sc_status|sc_substatus|sc_win32_status|time_taken|
+----------+-------------------+----+---------+------------------+------------+------+-----------+----+--------------------+--------------------+---------+------------+---------------+----------+
|2024-04-25|2024-04-25 06:22:57| ::1|      GET|/Products/Featured|           -|  8126|          -| ::1|Mozilla/5.0+(Wind...|http://localhost:...|      200|           0|              0|      6191|
|2024-04-25|2024-04-25 06:23:02| ::1|      GET|/Products/Featured|           -|  8126|          -| ::1|Mozilla/5.0+(Wind...|http://localhost:...|      200|           0|              0|      5012|
|2024-04-25|2024-04-

In [22]:
spark.sql("""SELECT min(time_taken),max(time_taken),avg(time_taken),cs_uri_stem 
from IISlogs2
where cs_uri_stem='/Products/Featured'
and time_taken > 0
group by cs_uri_stem 
""").show(5)

+---------------+---------------+-----------------+------------------+
|min(time_taken)|max(time_taken)|  avg(time_taken)|       cs_uri_stem|
+---------------+---------------+-----------------+------------------+
|           5012|          60119|47890.86111111111|/Products/Featured|
+---------------+---------------+-----------------+------------------+



In [23]:
spark.sql("""SELECT count(*) from IISlogs2 where time > '2024-04-25 06:22:57' and time < '2024-04-25 06:23:34'""").show(5)

+--------+
|count(1)|
+--------+
|       4|
+--------+

